In [1]:
import json
import math
import heapq
from collections import defaultdict, deque
import pandas as pd


In [2]:
def distancia_euclidiana(cidade1, cidade2):
    return math.sqrt(
        (cidade1['latitude'] - cidade2['latitude'])**2 +
        (cidade1['longitude'] - cidade2['longitude'])**2
    )

def construir_grafo(cidades, r):
    grafo = defaultdict(list)
    for cidade1 in cidades:
        for cidade2 in cidades:
            if cidade1 != cidade2:
                dist = distancia_euclidiana(cidade1, cidade2)
                if dist <= r:
                    grafo[cidade1['city']].append((cidade2['city'], dist))
    return grafo


In [3]:
def busca_a_estrela(grafo, cidades_dict, inicio, objetivo):
    fronteira = []
    heapq.heappush(fronteira, (0, inicio))
    caminho = {}
    custo_ate_agora = defaultdict(lambda: float('inf'))
    custo_ate_agora[inicio] = 0

    while fronteira:
        _, atual = heapq.heappop(fronteira)

        if atual == objetivo:
            rota = []
            while atual in caminho:
                rota.append(atual)
                atual = caminho[atual]
            rota.append(inicio)
            rota.reverse()
            return rota, custo_ate_agora[objetivo]

        for vizinho, custo in grafo[atual]:
            novo_custo = custo_ate_agora[atual] + custo
            if novo_custo < custo_ate_agora[vizinho]:
                caminho[vizinho] = atual
                custo_ate_agora[vizinho] = novo_custo
                prioridade = novo_custo + distancia_euclidiana(cidades_dict[vizinho], cidades_dict[objetivo])
                heapq.heappush(fronteira, (prioridade, vizinho))
    return None, float('inf')


In [4]:
def busca_bidirecional(grafo, inicio, objetivo):
    if inicio == objetivo:
        return [inicio], 0

    frente_inicio = deque([inicio])
    frente_objetivo = deque([objetivo])
    visitado_inicio = {inicio: None}
    visitado_objetivo = {objetivo: None}
    custo_inicio = {inicio: 0}
    custo_objetivo = {objetivo: 0}

    while frente_inicio and frente_objetivo:
        atual_inicio = frente_inicio.popleft()
        for vizinho, peso in grafo[atual_inicio]:
            if vizinho not in visitado_inicio:
                visitado_inicio[vizinho] = atual_inicio
                custo_inicio[vizinho] = custo_inicio[atual_inicio] + peso
                frente_inicio.append(vizinho)
            if vizinho in visitado_objetivo:
                return reconstruir_caminho(visitado_inicio, visitado_objetivo, vizinho, custo_inicio, custo_objetivo)

        atual_objetivo = frente_objetivo.popleft()
        for vizinho, peso in grafo[atual_objetivo]:
            if vizinho not in visitado_objetivo:
                visitado_objetivo[vizinho] = atual_objetivo
                custo_objetivo[vizinho] = custo_objetivo[atual_objetivo] + peso
                frente_objetivo.append(vizinho)
            if vizinho in visitado_inicio:
                return reconstruir_caminho(visitado_inicio, visitado_objetivo, vizinho, custo_inicio, custo_objetivo)

    return None, float('inf')

def reconstruir_caminho(visitado_inicio, visitado_objetivo, encontro, custo_inicio, custo_objetivo):
    caminho_inicio = []
    node = encontro
    while node:
        caminho_inicio.append(node)
        node = visitado_inicio[node]
    caminho_inicio.reverse()

    caminho_objetivo = []
    node = visitado_objetivo[encontro]
    while node:
        caminho_objetivo.append(node)
        node = visitado_objetivo[node]

    return caminho_inicio + caminho_objetivo, custo_inicio[encontro] + custo_objetivo[encontro]


In [5]:
with open("cities.json", "r") as f:
    cidades = json.load(f)

cidades_dict = {c['city']: c for c in cidades}
cenarios = [
    {"start": "Atlanta", "end": "Minneapolis", "r": 1.0},
    {"start": "Phoenix", "end": "Gilbert", "r": 2.5},
    {"start": "Baltimore", "end": "Jersey City", "r": 3.5}
]

resultados = []
for c in cenarios:
    grafo = construir_grafo(cidades, c["r"])
    caminho_a, custo_a = busca_a_estrela(grafo, cidades_dict, c["start"], c["end"])
    caminho_b, custo_b = busca_bidirecional(grafo, c["start"], c["end"])

    resultados.append({
        "Origem": c["start"],
        "Destino": c["end"],
        "Raio": c["r"],
        "A*_Caminho": caminho_a,
        "A*_Custo": round(custo_a, 2) if custo_a != float("inf") else "Sem solução",
        "Bidirecional_Caminho": caminho_b,
        "Bidirecional_Custo": round(custo_b, 2) if custo_b != float("inf") else "Sem solução"
    })

import pandas as pd
pd.DataFrame(resultados)


,Origem,Destino,Raio,A*_Caminho,A*_Custo,Bidirecional_Caminho,Bidirecional_Custo
0,Atlanta,Minneapolis,1.0,"[Atlanta, Smyrna, Brentwood, Dublin, Columbus,...",1.71,"[Atlanta, Smyrna, Brentwood, Concord, Lawrence...",2.40
1,Phoenix,Gilbert,2.5,"[Phoenix, Gilbert]",0.30,"[Phoenix, Gilbert]",0.30
2,Baltimore,Jersey City,3.5,"[Baltimore, Jersey City]",2.91,"[Baltimore, Jersey City]",2.91
